In [1]:
import numpy as np

In [2]:
import tensorflow.keras as keras

In [3]:
import os

In [4]:
test_dir=os.path.join(r'E:\aclImdb','test')

In [5]:
labels=[]
texts=[]
for label_type in ['neg','pos']:
    dir_name=os.path.join(test_dir,label_type)
    
    for file_name in os.listdir(dir_name):
        if file_name[-4:]=='.txt':
            f=open(os.path.join(dir_name,file_name),encoding='utf-8')
            texts.append(f.read())
            f.close
            if label_type=='neg':
                labels.append(0)
            else:
                labels.append(1)
        

In [6]:
len(texts)

17243

In [7]:
maxlen=100 #Cuts off review after 100 words
training_sample=10000 #Train on these many samples
validation_samples=7243
max_words=10000 #considers only top 10000 words

In [8]:
tokenizer=keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

In [9]:
word_index=tokenizer.word_index

In [17]:
data=keras.preprocessing.sequence.pad_sequences(sequences,maxlen=time_steps_unroll)

In [18]:
labels=np.asarray(labels)
print('Shape of Data Tensor:',data.shape)
print('Shape of Label Tensor:',labels.shape)

Shape of Data Tensor: (17243, 100)
Shape of Label Tensor: (17243,)


In [19]:
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]

In [13]:
embedding_index={}
f=open(os.path.join(r'E:\glove.6B','glove.6B.100d.txt'),encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=coefs
f.close()

In [14]:
embedding_dim=100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [16]:
batch_size=32
time_steps_unroll=100

In [20]:
training_data_review=data[:int(0.7*len(data))]
training_labels=labels[:int(0.7*len(labels))]
validation_data=data[int(0.7*len(data)):]
validation_labels=labels[int(0.7*len(data)):]

In [21]:
model=keras.Sequential()
model.add(keras.layers.Embedding(max_words,maxlen,input_length=time_steps_unroll,weights=[embedding_matrix]))


In [22]:
model.add(keras.layers.SimpleRNN(maxlen))

In [23]:
model.layers[0].trainable=False
model.add(keras.layers.Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               20100     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,020,201
Trainable params: 20,201
Non-trainable params: 1,000,000
_________________________________________________________________


In [25]:
history=model.fit(training_data_review,training_labels,epochs=50,batch_size=128)

Epoch 1/50
12070/12070 [==============================] - 8s 631us/step - loss: 0.6071 - acc: 0.7166
Epoch 2/50
12070/12070 [==============================] - 7s 558us/step - loss: 0.5853 - acc: 0.7271
Epoch 3/50
12070/12070 [==============================] - 7s 557us/step - loss: 0.5800 - acc: 0.7278
Epoch 4/50
12070/12070 [==============================] - 7s 559us/step - loss: 0.5739 - acc: 0.7286
Epoch 5/50
12070/12070 [==============================] - 7s 556us/step - loss: 0.5673 - acc: 0.7289
Epoch 6/50
12070/12070 [==============================] - 7s 556us/step - loss: 0.5603 - acc: 0.7321
Epoch 7/50
12070/12070 [==============================] - 7s 571us/step - loss: 0.5498 - acc: 0.7360
Epoch 8/50
12070/12070 [==============================] - 7s 560us/step - loss: 0.5388 - acc: 0.7421
Epoch 9/50
12070/12070 [==============================] - 7s 560us/step - loss: 0.5253 - acc: 0.7483
Epoch 10/50
12070/12070 [==============================] - 7s 563us/step - loss: 0.5078 - a